In [ ]:
#  LangChain Expression Language (LCEL)is used to chain components together

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['AZURE_OPENAI_KEY']=os.getenv("AZURE_OPENAI_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")



In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"), 
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
    api_key = os.getenv('AZURE_OPENAI_KEY'),  
    temperature=0.9  
)



In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B478DE6330>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B478F0F8C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Spanish"),
    HumanMessage(content="Hello How are you?")
]


result=model.invoke(messages)

In [4]:
result

AIMessage(content='Here are a couple of ways to say "Hello, how are you?" in Spanish:\n\n* **Hola, ¿cómo estás?** (informal, used with friends and family)\n* **Hola, ¿cómo está?** (formal, used with strangers or people you respect)\n\n\nLet me know if you\'d like to know how to say "how are you?" in other contexts!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 21, 'total_tokens': 106, 'completion_time': 0.154545455, 'prompt_time': 0.001333789, 'queue_time': 0.192456452, 'total_time': 0.155879244}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e2fa83b4-f53c-4bd0-b838-5f0fc9b6c509-0', usage_metadata={'input_tokens': 21, 'output_tokens': 85, 'total_tokens': 106})

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a couple of ways to say "Hello, how are you?" in Spanish:\n\n* **Hola, ¿cómo estás?** (informal, used with friends and family)\n* **Hola, ¿cómo está?** (formal, used with strangers or people you respect)\n\n\nLet me know if you\'d like to know how to say "how are you?" in other contexts!\n'

In [6]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)


'Here are a few ways to say "Hello, how are you?" in Spanish:\n\n* **Hola, ¿cómo estás?** (informal, to one person)\n* **Hola, ¿cómo están?** (informal, to multiple people)\n* **Buenos días, ¿cómo está?** (formal, to one person in the morning)\n* **Buenas tardes, ¿cómo está?** (formal, to one person in the afternoon/evening)\n\n\nLet me know if you\'d like more variations!\n'

In [7]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)
result=prompt.invoke({"language":"French","text":"Hello"})
result.to_messages()


[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [8]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'